<h1>АНАЛИЗ ДАННЫХ КОМПАНИИ NOZO</h1>

<h2>1. РАЗВЕДОЧНЫЙ АНАЛИЗ ДАННЫХ</h2>

<h3>1.1 ЗАГРУЗКА БИБЛИОТЕК И ЛИСТОВ ИСХОДНОГО ФАЙЛА</h3>

In [51]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import display_html

%matplotlib inline
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

from scipy.stats.stats import pearsonr
import seaborn as sns

In [52]:
xls = pd.ExcelFile('case-spasibo.xlsx')

df1 = pd.read_excel(xls, 'Points')
df2 = pd.read_excel(xls, 'Clients')
df3 = pd.read_excel(xls, 'New clients cohort')

<h3>1.2 ЛИСТ "POINTS"</h3>

Какой вид имеет данная таблица?

In [53]:
df1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,Month,"Accrual points , RUB","Write-off points, RUB",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,January 2019,60841048.91,394565298.501031,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,February 2019,81611851.159,491432996.503093,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,March 2019,73870409.4,457245205.354639,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,April 2019,63944494.45,436561210.329897,NaN,NaN,NaN,NaN,NaN,NaN


Необходимо удалить пустые столбцы (0, 4-9), а также изменить заголовки на элементы нулевой строки:

In [54]:
df1 = df1.drop(df1.columns[[0, 4, 5, 6, 7, 8, 9]], axis=1) # удаляем ненужные столбцы

In [55]:
df1.drop(labels=[0], axis=0, inplace=True) # удаляем нулевую строку

In [56]:
# переименовываем столбцы

df1.rename(columns={
    'Unnamed: 1': 'Months',
    'Unnamed: 2': 'Accrual points, RUB',
    'Unnamed: 3': 'Write-off points, RUB'
}, inplace=True)

In [57]:
df1.head()

,Months,"Accrual points, RUB","Write-off points, RUB"
1,January 2019,60841048.91,394565298.501031
2,February 2019,81611851.159,491432996.503093
3,March 2019,73870409.4,457245205.354639
4,April 2019,63944494.45,436561210.329897
5,May 2019,66284833.722,448355817.603093


In [58]:
df1.shape

(13, 3)

Таблица приведена к необходимому виду. Проверим, нет ли в ней пропусков:

In [59]:
df1.isna().sum() # сумма пропусков в каждом столбце

Months                   0
Accrual points, RUB      0
Write-off points, RUB    0
dtype: int64

Все ячейки заполнены. Последний шаг заключается в преобразовании числовых данных из типа str в float, чтобы с ними можно было производить операции:

In [60]:
df1['Accrual points, RUB'] = pd.to_numeric(df1['Accrual points, RUB'])
df1['Write-off points, RUB'] = pd.to_numeric(df1['Write-off points, RUB'])

df1.dtypes

Months                    object
Accrual points, RUB      float64
Write-off points, RUB    float64
dtype: object

На этом предобработка данных первого листа таблицы завершена.

<h3>1.3 ЛИСТ "CLIENTS"</h3>

Какой вид имеет данная таблица?

In [61]:
df2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Spasibo Clients,NaN,NaN,NaN,NaN,NOZO Clients,NaN,NaN,NaN,NaN
2,NaN,Month,Orders,GMV,Items,NaN,Month,Items,GMV,Orders,NaN
3,NaN,January 2019,1082112.9,1465070096.414447,1778499.2,NaN,January 2019,13993381.7,7404596108.857748,3335760.4,NaN
4,NaN,February 2019,978052.5,1551898714.676302,2255318.6,NaN,February 2019,18876522.9,7945350332.060995,2820148.7,NaN


Целесообразно сначала обработать всю таблицу по аналогии с предыдущим пунктом, а потом разделить на два фрейма: по клиентам "Спасибо" и по клиентам "NOZO" в целом

In [62]:
df2 = df2.drop(df2.columns[[0, 5, 10]], axis=1) # удаляем ненужные столбцы

In [63]:
df2.drop(labels=[0], axis=0, inplace=True) # удаляем нулевую строку

In [64]:
# переименовываем столбцы

# _s означает, что столбцы относятся к 'Spasibo clients'
# _n означает, что столбцы относятся к 'NOZO clients'

df2.rename(columns={
    'Unnamed: 1': 'Month_s',
    'Unnamed: 2': 'Orders_s',
    'Unnamed: 3': 'GMV_s',
    'Unnamed: 4': 'Items_s',
    
    'Unnamed: 6': 'Month_n',
    'Unnamed: 7': 'Items_n',
    'Unnamed: 8': 'GMV_n',
    'Unnamed: 9': 'Orders_n'
}, inplace=True)

In [65]:
df2.drop(labels=[1, 2], axis=0, inplace=True) # удаляем первые две строки

In [66]:
df2_1 = df2.loc[:, 'Month_s':'Items_s'] # вырезаем датафрейм по клиентам "Спасибо"
df2_1.head(2)

,Month_s,Orders_s,GMV_s,Items_s
3,January 2019,1082112.9,1465070096.414447,1778499.2
4,February 2019,978052.5,1551898714.676302,2255318.6


In [67]:
df2_2 = df2.loc[:, 'Month_n':'Orders_n'] # вырезаем датафрейм по клиентам "NOZO"
df2_2.head(2)

,Month_n,Items_n,GMV_n,Orders_n
3,January 2019,13993381.7,7404596108.857748,3335760.4
4,February 2019,18876522.9,7945350332.060995,2820148.7


In [68]:
df2_1.shape, df2_2.shape

((12, 4), (12, 4))

Датафреймы приведены к необходимому виду. Проверим, нет ли пропусков в каждом из них:

In [69]:
df2_1.isna().sum() # сумма пропусков в каждом столбце

Month_s     0
Orders_s    0
GMV_s       0
Items_s     0
dtype: int64

In [70]:
df2_2.isna().sum() # сумма пропусков в каждом столбце

Month_n     0
Items_n     0
GMV_n       0
Orders_n    0
dtype: int64

Все ячейки заполнены. Последний шаг - преобразование числовых данных из типа str в float, чтобы с ними можно было производить операции:

In [71]:
df2_1['Orders_s'] = pd.to_numeric(df2_1['Orders_s'])
df2_1['GMV_s'] = pd.to_numeric(df2_1['GMV_s'])
df2_1['Items_s'] = pd.to_numeric(df2_1['Items_s'])

df2_1.dtypes

Month_s      object
Orders_s    float64
GMV_s       float64
Items_s     float64
dtype: object

In [72]:
df2_2['Orders_n'] = pd.to_numeric(df2_2['Orders_n'])
df2_2['GMV_n'] = pd.to_numeric(df2_2['GMV_n'])
df2_2['Items_n'] = pd.to_numeric(df2_2['Items_n'])

df2_2.dtypes

Month_n      object
Items_n     float64
GMV_n       float64
Orders_n    float64
dtype: object

Датафреймы полностью готовы к обработке.

<h3>1.4 ЛИСТ "NEW CLIENTS COHORT"</h3>

Какой вид имеет таблица?

In [73]:
df3.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Spasibo Clients,- 1st order was made with Spasibo points,NaN,NaN,NaN,NaN,All NOZO Clients,NaN,- new clients,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Client Base for April 2018,NaN,NaN,NaN,NaN,NaN,Client Base for April 2018,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,56288.7,NaN,NaN,NaN,NaN,NaN,343546.2,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Month,Year,Sum of Retained_Clients,GMV,Orders,NaN,Month,Year,Sum of Retained_Clients,GMV,Orders,NaN,NaN
8,NaN,NaN,5,2018,8255.2,23177561.9014,10749.1,NaN,5,2018,39021.8,154945358.020504,58374.6,NaN,NaN
9,NaN,NaN,6,2018,6563.7,19868381.0898,9596.5,NaN,6,2018,30057.7,116976178.866385,43887.2,NaN,NaN


Данная таблица содержит следующие комментарии:

1. Клиенты "Спасибо" - первый заказ был сделан с использованием бонусов "Спасибо"
2. Клиентская база к апрелю 2018г. - 56288.7
3. Все клиенты NOZO - новые клиенты
4. Клиентская база к апрелю 2018г. - 343546.2

Эти комментарии необходимы в качестве пояснений, но будут лишними в процессе обработки данных, поэтому в дальнейшем будут удалены.

По аналогии с предыдущим пунктом, обработаем всю таблицу, а потом разделим на два фрейма: по клиентам "Спасибо" и по клиентам "NOZO":

In [74]:
df3 = df3.drop(df3.columns[[0, 1, 7, 13, 14]], axis=1) # удаляем ненужные столбцы

In [75]:
df3.drop(labels=[0, 1, 2, 3, 4, 5, 6], axis=0, inplace=True) # удаляем ненужные строки

In [76]:
# переименовываем столбцы

# _s означает, что столбцы относятся к 'Spasibo clients'
# _n означает, что столбцы относятся к 'NOZO clients'

df3.rename(columns={
    'Unnamed: 2': 'Month_s',
    'Unnamed: 3': 'Year_s',
    'Unnamed: 4': 'Sum_of_Retained_Clients_s',
    'Unnamed: 5': 'GMV_s',
    'Unnamed: 6': 'Orders_s',
    
    'Unnamed: 8': 'Month_n',
    'Unnamed: 9': 'Year_n',
    'Unnamed: 10': 'Sum_of_Retained_Clients_n',
    'Unnamed: 11': 'GMV_n',
    'Unnamed: 12': 'Orders_n'
}, inplace=True)

In [77]:
df3.drop(labels=[7], axis=0, inplace=True) # удаляем первую строку

In [78]:
df3_1 = df3.loc[:, 'Month_s':'Orders_s'] # вырезаем датафрейм по клиентам "Спасибо"
df3_1.head(2)

,Month_s,Year_s,Sum_of_Retained_Clients_s,GMV_s,Orders_s
8,5,2018,8255.2,23177561.9014,10749.1
9,6,2018,6563.7,19868381.0898,9596.5


In [79]:
df3_2 = df3.loc[:, 'Month_n':'Orders_n'] # вырезаем датафрейм по клиентам NOZO
df3_2.head(2)

,Month_n,Year_n,Sum_of_Retained_Clients_n,GMV_n,Orders_n
8,5,2018,39021.8,154945358.020504,58374.6
9,6,2018,30057.7,116976178.866385,43887.2


In [80]:
df3_1.shape, df3_2.shape

((16, 5), (16, 5))

Датафреймы приведены к необходимому виду. Проверим, нет ли пропусков в каждом из них:

In [81]:
df3_1.isna().sum() # сумма пропусков в каждом столбце

Month_s                      0
Year_s                       0
Sum_of_Retained_Clients_s    0
GMV_s                        0
Orders_s                     0
dtype: int64

In [82]:
df3_2.isna().sum() # сумма пропусков в каждом столбце

Month_n                      0
Year_n                       0
Sum_of_Retained_Clients_n    0
GMV_n                        0
Orders_n                     0
dtype: int64

Все ячейки заполнены. Осталось преобразовать числовые данные из типа str в float, чтобы с ними можно было производить операции:

In [83]:
df3_1['Month_s'] = pd.to_numeric(df3_1['Month_s'])
df3_1['Year_s'] = pd.to_numeric(df3_1['Year_s'])
df3_1['Sum_of_Retained_Clients_s'] = pd.to_numeric(df3_1['Sum_of_Retained_Clients_s'])
df3_1['GMV_s'] = pd.to_numeric(df3_1['GMV_s'])
df3_1['Orders_s'] = pd.to_numeric(df3_1['Orders_s'])

df3_1.dtypes

Month_s                        int64
Year_s                         int64
Sum_of_Retained_Clients_s    float64
GMV_s                        float64
Orders_s                     float64
dtype: object

In [84]:
df3_2['Month_n'] = pd.to_numeric(df3_2['Month_n'])
df3_2['Year_n'] = pd.to_numeric(df3_2['Year_n'])
df3_2['Sum_of_Retained_Clients_n'] = pd.to_numeric(df3_2['Sum_of_Retained_Clients_n'])
df3_2['GMV_n'] = pd.to_numeric(df3_2['GMV_n'])
df3_2['Orders_n'] = pd.to_numeric(df3_2['Orders_n'])

df3_2.dtypes

Month_n                        int64
Year_n                         int64
Sum_of_Retained_Clients_n    float64
GMV_n                        float64
Orders_n                     float64
dtype: object

На этом разведочный анализ данных завершен. Все датафреймы готовы к дальнейшей обработке.

<h2>2. ВИЗУАЛИЗАЦИЯ ИСХОДНЫХ ДАННЫХ</h2>

<h3>2.1 СОЗДАНИЕ ПОНЯТНЫХ ПЕРЕМЕННЫХ</h3>

Чтобы в процессе анализа не запутаться в названиях переменных, соответствующих каждому датафрейму, переименуем их:

In [85]:
points = df1
clients_s = df2_1
clients_n = df2_2
cohort_s = df3_1
cohort_n = df3_2

Также в процессе разведочного анализа было выяснено, что в одной из таблиц содержались комментарии по размеру клиентских баз. Занесем их в отдельные переменные:

In [86]:
cl_base_s = 56288.7 # клиентская база "Спасибо" к апрелю 2018г.
cl_base_n = 343546.2 # клиентская база NOZO к апрелю 2018г.

<h3>2.2 ВЫВОД ТАБЛИЦ</h3>

Для вывода таблиц бок-о-бок нам потребуется функция:

In [87]:
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

Источник кода: https://bit.ly/3bVgbdc

In [88]:
points

,Months,"Accrual points, RUB","Write-off points, RUB"
1,January 2019,6.084105e+07,3.945653e+08
2,February 2019,8.161185e+07,4.914330e+08
3,March 2019,7.387041e+07,4.572452e+08
4,April 2019,6.394449e+07,4.365612e+08
5,May 2019,6.628483e+07,4.483558e+08
6,June 2019,4.894842e+07,4.098297e+08
7,July 2019,5.856699e+07,4.462786e+08
8,August 2019,9.323797e+07,4.890571e+08
9,September 2019,7.823140e+07,5.030062e+08
10,October 2019,9.373171e+07,5.898544e+08


<b>Accrual points, RUB</b> - начисленные баллы в соответствующем месяце<br>
<b>Write-off points, RUB</b> - списанные баллы в соответствующем месяце

In [89]:
display_side_by_side(clients_s, clients_n)

,Month_s,Orders_s,GMV_s,Items_s
3,January 2019,1082112.9,1.465070e+09,1778499.2
4,February 2019,978052.5,1.551899e+09,2255318.6
5,March 2019,928638.6,1.480984e+09,2126120.3
6,April 2019,1061719.7,1.536529e+09,2396394.8
7,May 2019,1125077.0,1.735549e+09,2792319.7
8,June 2019,956285.7,1.733175e+09,3094501.5
9,July 2019,1007093.6,1.936924e+09,3673375.3
10,August 2019,1108857.3,2.049963e+09,4088129.4
11,September 2019,1187873.3,2.067659e+09,4109483.1
12,October 2019,1425886.9,2.496168e+09,4991752.5


<b>Orders_s</b> - количество заказов, в которых был списан хотя бы один бонус "Спасибо", в соответствующем месяце<br>
<b>Orders_n</b> - количество заказов в целом по компании в соответствующем месяце<br>
<b>Items</b> - количество проданных товаров с этих заказов в соответствующем месяце<br>
<b>GMV</b> - совокупный объем продаж в соответствующем месяце

In [90]:
display_side_by_side(cohort_s, cohort_n) 
# когорты клиентов, сделавших свой первый заказ с бонусами "Спасибо" в апреле-18 и их последующий перформанс

,Month_s,Year_s,Sum_of_Retained_Clients_s,GMV_s,Orders_s
8,5,2018,8255.2,2.317756e+07,10749.1
9,6,2018,6563.7,1.986838e+07,9596.5
10,7,2018,5919.4,1.724444e+07,7820.0
11,8,2018,6223.7,1.677418e+07,7723.1
12,9,2018,6696.3,1.821825e+07,8705.7
13,10,2018,7364.4,1.891271e+07,9751.2
14,11,2018,7743.5,2.210852e+07,11616.1
15,12,2018,9100.1,2.536961e+07,12807.8
16,1,2019,7196.1,3.186856e+07,14919.2
17,2,2019,7522.5,2.169215e+07,11583.8


<b>Sum_of_Retained_Clients</b> - количество оставшихся клиентов<br>
<b>GMV</b> - совокупный объем продаж в соответствующем месяце<br>
<b>Orders</b> - количество заказов, в которых был списан хотя бы один бонус "Спасибо", в соответствующем месяце

Теперь имеется представление о том, как выглядят таблицы, а также они имеют понятные названия. Можно переходить непосредственно к анализу данных.

<h2>3. ВЫДВИЖЕНИЕ И ТЕСТИРОВАНИЕ ГИПОТЕЗ</h2>

<h3>3.1 ГИПОТЕЗА №1: ОПТИМАЛЬНОЕ РЕШЕНИЕ - ОТКАЗ ОТ УЧАСТИЯ В ПРОГРАММЕ "СБЕРСПАСИБО"</h3>

Чтобы подтвердить или опровергнуть данную гипотезу, необходимо проанализировать данные и ответить на следующие вопросы:

1. Оказывает ли значительное влияние возможность частично оплатить заказ с использованием бонусов "Спасибо" на желание покупателей приобрести товары в NOZO?
2. Зависит ли увеличение/уменьшение GMV от повышения/понижения числа клиентов, пользующихся бонусами "Спасибо" и их лояльности? Как именно?
3. Сколько тратит компания NOZO на данную программу?

<b>3.1.1. Оказывает ли значительное влияние возможность частично оплатить заказ с использованием бонусов "Спасибо" на желание покупателей приобрести товар в NOZO?</b>

Для ответа на данный вопрос необходимо выяснить, как коррелируют между собой две величины: 1) увеличение/уменьшение доли заказов, в которых был списан хотя бы один бонус "Спасибо", в общем числе заказов от месяца к месяцу в 2019г.; 2) увеличение/уменьшение общего показателя GMV от месяца к месяцу в 2019г.

In [91]:
# сначала создадим переменные для величин 1 и 2

ratio1 = clients_s['Orders_s'] / clients_n['Orders_n'] # доли заказов с бонусами "Спасибо" в общем числе заказов

ratio1_0 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                         'Ratio1_0': ratio1.loc[3:13]}) # срез с января по ноябрь

ratio1_1 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                         'Ratio1_1': ratio1.loc[4:14]}) # срез с февраля по декабрь

ratios1 = pd.merge(ratio1_1, ratio1_0, on='Index', how='left')
val_1 = pd.to_numeric(ratios1['Ratio1_1']) - pd.to_numeric(ratios1['Ratio1_0']) # изменение доли заказов от месяца к месяцу

gmv2 = clients_n['GMV_n'] # GMV в целом по компании NOZO

gmv2_0 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                         'GMV2_0': gmv2.loc[3:13]}) # срез с января по ноябрь

gmv2_1 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                         'GMV2_1': gmv2.loc[4:14]}) # срез с февраля по декабрь

gmvs2 = pd.merge(gmv2_1, gmv2_0, on='Index', how='left')
val_2 = pd.to_numeric(gmvs2['GMV2_1']) - pd.to_numeric(gmvs2['GMV2_0']) # изменение GMV от месяца к месяцу

In [92]:
# найдем коэффициент корреляции между данными величинами

print(pearsonr(val_1, val_2)[0])

0.12085607177945495


<b>Вывод:</b> коэффициент корреляции является положительным числом, но по модулю близок к нулю. Таким образом, изменение GMV прямо зависит от изменения доли заказов, оплаченных в том числе с помощью бонусов "Спасибо", в общем числе заказов, но эта зависимость крайне несущественна, то есть на желание потребителя приобрести товары именно в NOZO оказывают влияние преимущественно другие факторы. Из этого наблюдения можно сделать предположение о том, что отказ компанией NOZO от участия в программе "СберСпасибо" незначительно повлияет на общий объем оборота товаров.

<b>3.1.2. Зависит ли увеличение/уменьшение GMV от повышения/понижения степени лояльности клиентов? Как именно?</b>

Теперь проанализируем когорты клиентов, по которым предоставлены данные. Для ответа на вопрос необходимо рассчитать величину и направление корреляции между следующими величинами: 1) изменение доли оставшихся новых клиентов, пользующихся программой "СберСпасибо", в общей когорте "клиентов-новичков" от месяца к месяцу с мая 2018г. по август 2019г.; 2) изменение GMV на пространстве всех "клиентов-новичков" компании NOZO от месяца к месяцу с мая 2018г. по август 2019г..

In [93]:
# сначала создадим переменные для величин 1 и 2

clients_ret = cohort_s['Sum_of_Retained_Clients_s'] / cohort_n['Sum_of_Retained_Clients_n'] 
# доля оставшихся клиентов, пользующихся программой "СберСпасибо", в общем количестве клиентов

clients_ret_0 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                         'Clients_ret_0': clients_ret.loc[8:22]}) # срез с мая 2018 по июль 2019

clients_ret_1 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                         'Clients_ret_1': clients_ret.loc[9:23]}) # срез с июня 2018 по август 2019

clients_ret_table = pd.merge(clients_ret_1, clients_ret_0, on='Index', how='left')
feat_1 = pd.to_numeric(clients_ret_table['Clients_ret_1']) - pd.to_numeric(clients_ret_table['Clients_ret_0'])
# изменение доли клиентов от месяца к месяцу

cohort_gmv = cohort_n['GMV_n'] # GMV в целом по когорте "клиентов-новичков" NOZO

cohort_gmv_0 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                         'Cohort_gmv_0': cohort_gmv.loc[8:22]}) # срез с мая 2018 по июль 2019

cohort_gmv_1 = pd.DataFrame({'Index': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
                         'Сohort_gmv_1': cohort_gmv.loc[9:23]}) # срез с июня 2018 по август 2019

cohorts_gmv = pd.merge(cohort_gmv_1, cohort_gmv_0, on='Index', how='left')
feat_2 = pd.to_numeric(cohorts_gmv['Сohort_gmv_1']) - pd.to_numeric(cohorts_gmv['Cohort_gmv_0']) 
# изменение GMV от месяца к месяцу

In [94]:
# найдем коэффициент корреляции между данными величинами

print(pearsonr(feat_1, feat_2)[0])

0.17047145079854475


<b>Вывод:</b> снова получено значение положительное и близкое к нулю по модулю, следовательно, изменение GMV компании NOZO практически не зависит от лояльности клиентов, которые предпочитают оплачивать заказы, используя бонусы "Спасибо". Таким образом, на лояльность клиентов в целом влияют другие факторы, а не непосредственно участие компании NOZO в программе от Сбербанка.

<b>3.1.3. Сколько тратит компания NOZO на данную программу?</b>

По условию, компания NOZO несет два вида расходов в рамках данной программы:<br>
1) комиссия от суммы, списанной в качестве бонусов покупателями;<br>
2) начисления бонусов покупателям от денежной части чека

Чтобы расчитать расходы компании на программу, нужно учесть обе статьи.

In [95]:
# средняя ставка комиссии Сбербанку:
ratio_x = ((points['Write-off points, RUB'][13] / points['Accrual points, RUB'][13]))/points.shape[0]

if ratio_x <= 7:
    fee = 0.03
elif 7 < ratio_x <= 9:
    fee = 0.04
elif 9 < ratio_x <= 11:
    fee = 0.05
else:
    fee = 0.07
    
# расходы по первой статье
expense_1 = (points['Write-off points, RUB'][13]) * fee

# расходы по второй статье

# минимальное значение
expense_2_min = ((points['Accrual points, RUB'])[13]) * 0.01
# максимальное значение
expense_2_max = ((points['Accrual points, RUB'])[13]) * 0.03

print('Компания NOZO тратит на программу "Спасибо" от', round(expense_1+expense_2_min) // 1000000, 'до', round(expense_1+expense_2_max) // 1000000, 'миллионов рублей в год')

Компания NOZO тратит на программу "Спасибо" от 199 до 219 миллионов рублей в год


<b>Вывод</b>

Гипотеза о том, что оптимальным решением является отказ от программы партнерства со Сбербанком, подтверждается. Компания NOZO может тратить до 219 миллионов рублей в год на данную программу, но данные расходы не оправдываются в связи с тем, что:<br>
а) возможность частично оплатить заказ с использованием баллов "Спасибо" не оказывает существенное влияние на желание покупателей приобрести товар в NOZO;<br>
б) изменение GMV компании NOZO практически не зависит от лояльности клиентов, предпочитающих оплачивать заказы, используя баллы "Спасибо"

<h3>3.2. ГИПОТЕЗА №2: ИЗМЕНЕНИЕ СТАВКИ НАЧИСЛЕНИЙ ПОМОЖЕТ ПОЛУЧИТЬ Б<font color='deeppink'>О</font>ЛЬШУЮ ВЫГОДУ</h3>

Чтобы проверить, изменить ли ставку начислений клиентам (т.е. остаться партнером программы, но изменить условия), необходимо выяснить, стоит ли ее увеличить или уменьшить.

<b>3.2.1. Уменьшение ставки начислений</b>

Если ставка начислений сокращается (соответственно, уменьшается переменная expense_2_min (expense_2_max)), то возрастает коэффициент, равный отношению списанных баллов к начисленным, а значит увеличивается коммисия Сбербанку. Таким образом, данный вариант удовлетворит Сбер, но под вопросом выгода компании NOZO.

Предположим, "потолок" начисленных баллов теперь составляет не 3%, а 2% (т.е. NOZO теперь может начислить клиенту 1 или 2% от денежной части). Тогда расчеты будут выглядеть следующим образом:

In [96]:
# измененная сумма начисленных баллов
points['Accrual points, RUB'] = points['Accrual points, RUB'] * 2 / 3

# средняя ставка комиссии Сбербанку:
ratio_x_1 = ((points['Write-off points, RUB'][13] / points['Accrual points, RUB'][13]))/points.shape[0]

if ratio_x_1 <= 7:
    fee_1 = 0.03
elif 7 < ratio_x_1 <= 9:
    fee_1 = 0.04
elif 9 < ratio_x_1 <= 11:
    fee_1 = 0.05
else:
    fee_1 = 0.07
    
if fee_1 == fee:
    print('Комиссия Сбербанку не изменилась')
else:
    print('Комиссия Сбербанку изменилась')

Комиссия Сбербанку не изменилась


Таким образом, если понизить ставку начислений клиентам с денежной части чека, для Сбербанка это не будет являться убытком. Проверим, насколько данное решение поможет сократить расходы компании NOZO:

In [97]:
# расходы по первой статье
expense_1_1 = (points['Write-off points, RUB'][13]) * fee_1

# максимальные расходы по второй статье
expense_2_max_1 = ((points['Accrual points, RUB'])[13]) * 0.02

print('Компания NOZO может тратить на программу "Спасибо" от', round(expense_1_1+expense_2_min) // 1000000, 'до', round(expense_1_1+expense_2_max_1) // 1000000, 'миллионов рублей в год')

Компания NOZO может тратить на программу "Спасибо" от 199 до 202 миллионов рублей в год


Заметно, что изменение ставки начислений поможет сократить расходы компании NOZO в среднем на 17 миллионов рублей в год, так как при определенном размере ставки комиссия Сбербанку по-прежнему не увеличивается.

<b>3.2.1. Увеличение ставки начислений</b>

Если ставка начислений растет, то растут и расходы компании NOZO по начислению баллов клиентам, но сокращается комиссия, которую компания обязана выплатить Сбербанку. В этом случае банк может не устроить такая сделка, поэтому у компании NOZO появятся новые барьеры.

<b>Вывод</b>

Оптимальное решение - оставить максимальную ставку начислений на уровне 2%, так как:<br>
а) согласно выводам из пункта 3.1, изменение GMV компании NOZO практически не зависит от лояльности клиентов, предпочитающих оплачивать заказы, используя баллы "Спасибо" и возможность частично оплатить заказ с использованием баллов "Спасибо" не оказывает существенное влияние на желание покупателей приобрести товар в NOZO;<br>
б) понижение верхнего порога ставки до меньшего значения несет достаточно большие риски, так как может отрицательно сказаться на лояльности клиентов и повлечь за собой негативные последствия<br>
в) такое решение поможет компании NOZO сократить расходы на программу